In [ ]:
import numpy as np
import pandas as pd
import string
from string import punctuation
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.utils import shuffle
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Womens-Clothing-E-Commerce-Reviews.csv')

In [ ]:
df.head(5)

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [ ]:
df = df.drop(['Unnamed: 0', 'Title', 'Positive Feedback Count'], axis = 1)

In [ ]:
df.dropna(inplace=True)

In [ ]:
df['Rating_Polarity'] = df['Rating'].apply(lambda rating : 'Positive' if(rating > 3) else ('Neutral' if(rating) == 3 else 'Negative' ))
df.shape

(22628, 9)

In [ ]:
df_positive  = df[df['Rating_Polarity'] ==  'Positive'] [0:8000]
df_neutral = df[df['Rating_Polarity'] == 'Neutral']
df_negative = df[df['Rating_Polarity'] == 'Negative']

In [ ]:
df_Neutral_over = df_neutral.sample(8000, replace=True)
df_Negative_over = df_negative.sample(8000, replace=True)

In [ ]:
df = pd.concat([df_positive, df_Neutral_over, df_Negative_over], axis = 0)
df

,Clothing ID,Age,Review Text,Rating,Recommended IND,Division Name,Department Name,Class Name,Rating_Polarity
0,767,33,Absolutely wonderful - silky and sexy and comf...,4,1,Initmates,Intimate,Intimates,Positive
1,1080,34,Love this dress! it's sooo pretty. i happene...,5,1,General,Dresses,Dresses,Positive
3,1049,50,"I love, love, love this jumpsuit. it's fun, fl...",5,1,General Petite,Bottoms,Pants,Positive
4,847,47,This shirt is very flattering to all due to th...,5,1,General,Tops,Blouses,Positive
6,858,39,I aded this in my basket at hte last mintue to...,5,1,General Petite,Tops,Knits,Positive
...,...,...,...,...,...,...,...,...,...
6938,1084,33,I was so excited to get this dress since it lo...,1,0,General Petite,Dresses,Dresses,Negative
17271,861,30,I am not in love with this top and will be ret...,1,0,General,Tops,Knits,Negative
20240,868,30,"Just a ginormous, shapeless shirt. runs extrem...",1,0,General,Tops,Knits,Negative
7489,836,37,So bummed. i was really hoping this top would ...,2,0,General,Tops,Blouses,Negative


In [ ]:
stpword = stopwords.words('english')
def get_text_processing(text):
    no_punctuation = [char for char in text if char not in string.punctuation]
    no_punctuation = ''.join(no_punctuation)
    return ' '.join([word for word in no_punctuation.split() if word.lower() not in stpword])

In [ ]:
df['review'] = df['Review Text'].apply(get_text_processing)
df.head()

,Clothing ID,Age,Review Text,Rating,Recommended IND,Division Name,Department Name,Class Name,Rating_Polarity,review
0,767,33,Absolutely wonderful - silky and sexy and comf...,4,1,Initmates,Intimate,Intimates,Positive,Absolutely wonderful silky sexy comfortable
1,1080,34,Love this dress! it's sooo pretty. i happene...,5,1,General,Dresses,Dresses,Positive,Love dress sooo pretty happened find store im ...
3,1049,50,"I love, love, love this jumpsuit. it's fun, fl...",5,1,General Petite,Bottoms,Pants,Positive,love love love jumpsuit fun flirty fabulous ev...
4,847,47,This shirt is very flattering to all due to th...,5,1,General,Tops,Blouses,Positive,shirt flattering due adjustable front tie perf...
6,858,39,I aded this in my basket at hte last mintue to...,5,1,General Petite,Tops,Knits,Positive,aded basket hte last mintue see would look lik...


In [ ]:
df = df[['review', 'Rating_Polarity']]
df.head()

,review,Rating_Polarity
0,Absolutely wonderful silky sexy comfortable,Positive
1,Love dress sooo pretty happened find store im ...,Positive
3,love love love jumpsuit fun flirty fabulous ev...,Positive
4,shirt flattering due adjustable front tie perf...,Positive
6,aded basket hte last mintue see would look lik...,Positive


In [ ]:
one_hot = pd.get_dummies(df["Rating_Polarity"])
df.drop(["Rating_Polarity"], axis=1, inplace=True)
df = pd.concat([df, one_hot], axis=1)
df.head()

<ipython-input-13-248a232571e1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(["Rating_Polarity"], axis=1, inplace=True)


,review,Negative,Neutral,Positive
0,Absolutely wonderful silky sexy comfortable,0,0,1
1,Love dress sooo pretty happened find store im ...,0,0,1
3,love love love jumpsuit fun flirty fabulous ev...,0,0,1
4,shirt flattering due adjustable front tie perf...,0,0,1
6,aded basket hte last mintue see would look lik...,0,0,1


In [ ]:
print(one_hot)

       Negative  Neutral  Positive
0             0        0         1
1             0        0         1
3             0        0         1
4             0        0         1
6             0        0         1
...         ...      ...       ...
6938          1        0         0
17271         1        0         0
20240         1        0         0
7489          1        0         0
8730          1        0         0

[24000 rows x 3 columns]


In [ ]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [
                layers.Dense(ff_dim, activation="relu"),
                layers.Dense(embed_dim),
            ]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [ ]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [ ]:
vocab_size = 20000
maxlen = 63

In [ ]:
X = df["review"].values
y = df.drop("review", axis=1).values

max_features = 20000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)
X = pad_sequences(X, maxlen=63)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42
)

print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(16800, 63) (16800, 3)
(7200, 63) (7200, 3)


In [ ]:
embed_dim = 100
num_heads = 4
ff_dim = 200

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(3, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.compile(
    optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"]
)
history = model.fit(
    X_train, y_train, batch_size=128, epochs=15, validation_data=(X_test, y_test)
)

Epoch 1/15
132/132 [==============================] - 85s 596ms/step - loss: 1.0458 - accuracy: 0.4126 - val_loss: 0.7963 - val_accuracy: 0.6060
Epoch 2/15
132/132 [==============================] - 80s 603ms/step - loss: 0.6562 - accuracy: 0.7028 - val_loss: 0.5736 - val_accuracy: 0.7589
Epoch 3/15
132/132 [==============================] - 77s 583ms/step - loss: 0.3985 - accuracy: 0.8487 - val_loss: 0.4363 - val_accuracy: 0.8457
Epoch 4/15
132/132 [==============================] - 75s 568ms/step - loss: 0.2425 - accuracy: 0.9180 - val_loss: 0.4424 - val_accuracy: 0.8572
Epoch 5/15
132/132 [==============================] - 78s 589ms/step - loss: 0.1626 - accuracy: 0.9479 - val_loss: 0.4400 - val_accuracy: 0.8829
Epoch 6/15
132/132 [==============================] - 77s 587ms/step - loss: 0.1216 - accuracy: 0.9607 - val_loss: 0.4621 - val_accuracy: 0.8840
Epoch 7/15
132/132 [==============================] - 79s 601ms/step - loss: 0.0926 - accuracy: 0.9708 - val_loss: 0.5276 - val_ac

In [ ]:
model.save("/content/drive/MyDrive/sentu2.h5")

In [ ]:
text = ["The quality is bad. I hate the product."]
text = tokenizer.texts_to_sequences(text)
text = pad_sequences(text, maxlen=63, dtype='int32', value=0)
sentiment = model.predict(text,batch_size=1,verbose = 2)[0]

print(sentiment)

if (np.argmax(sentiment) == 0):
  print("negative")
elif (np.argmax(sentiment) == 1):
  print("neutral")
else:
  print("positive")

1/1 - 0s - 21ms/epoch - 21ms/step
[1.4910898e-05 9.9992979e-01 5.5342778e-05]
neutral
